# Logistic Regression

Let's see an example of how to run a logistic regression with Python and Spark! This is documentation example, we will quickly run through this and then show a more realistic example, afterwards, you will have another consulting project!

In [1]:
import findspark
findspark.init('/home/shashank/spark-2.3.2-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('logregdoc').getOrCreate()

In [2]:
from pyspark.ml.classification import LogisticRegression

In [5]:
# Load training data
training = spark.read.format("libsvm").load("sample_libsvm_data.txt")

training.createOrReplaceTempView('train_sql')

lr = LogisticRegression()

# Fit the model
lrModel = lr.fit(training)

trainingSummary = lrModel.summary

In [19]:
trainingSummary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [6]:
trainingSummary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111056...|       1.0|
|  1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126979...|       1.0|
|  1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170303...|       1.0|
|  1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200604...|       1.0|
|  0.0|(692,[129,130,131...|[19.8506078990277...|[0.99999999760673...|       0.0|
|  1.0|(692,[158,159,160...|[-20.337256674833...|[1.47109814695581...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.595579753418...|[3.08850168102631...|       1.0|
|  0.0|(692,[154,155,156...|[19.2708803215613...|[0.99999999572670...|       0.0|
|  0.0|(692,[127

In [7]:
# May change soon!
from pyspark.mllib.evaluation import MulticlassMetrics

In [8]:
lrModel.evaluate(training)

In [10]:
# Usually would do this on a separate test set!
predictionAndLabels = lrModel.evaluate(training)

In [11]:
predictionAndLabels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111056...|       1.0|
|  1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126979...|       1.0|
|  1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170303...|       1.0|
|  1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200604...|       1.0|
|  0.0|(692,[129,130,131...|[19.8506078990277...|[0.99999999760673...|       0.0|
|  1.0|(692,[158,159,160...|[-20.337256674833...|[1.47109814695581...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.595579753418...|[3.08850168102631...|       1.0|
|  0.0|(692,[154,155,156...|[19.2708803215613...|[0.99999999572670...|       0.0|
|  0.0|(692,[127

In [12]:
predictionAndLabels = predictionAndLabels.predictions.select('label','prediction')

In [13]:
predictionAndLabels.show()

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
+-----+----------+
only showing top 20 rows



## Evaluators

Evaluators will be a very important part of our pipline when working with Machine Learning, let's see some basics for Logistic Regression, useful links:

https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.BinaryClassificationEvaluator

https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.MulticlassClassificationEvaluator

In [14]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator

In [15]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label')

In [16]:
# For multiclass
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label',
                                             metricName='accuracy')

In [17]:
acc = evaluator.evaluate(predictionAndLabels)

In [18]:
acc

1.0

Okay let's move on see some more examples!

In [22]:
my_data = spark.read.format("libsvm").load("sample_libsvm_data.txt")

In [23]:
train,test = my_data.randomSplit([0.7,0.3])

In [51]:
final_model = LogisticRegression()
fit_final = final_model.fit(train)

In [52]:
fit_final.summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[95,96,97,12...|[21.6255121960616...|[0.99999999959434...|       0.0|
|  0.0|(692,[98,99,100,1...|[24.9374198766601...|[0.99999999998521...|       0.0|
|  0.0|(692,[121,122,123...|[20.7944129935967...|[0.99999999906867...|       0.0|
|  0.0|(692,[123,124,125...|[27.1106774418274...|[0.99999999999831...|       0.0|
|  0.0|(692,[123,124,125...|[28.6366800219400...|[0.99999999999963...|       0.0|
|  0.0|(692,[123,124,125...|[21.1551234496019...|[0.99999999935069...|       0.0|
|  0.0|(692,[124,125,126...|[22.0389935422573...|[0.99999999973172...|       0.0|
|  0.0|(692,[124,125,126...|[30.1704945064754...|[0.99999999999992...|       0.0|
|  0.0|(692,[124,125,126...|[29.3503982660057...|[0.99999999999982...|       0.0|
|  0.0|(692,[124

In [54]:
p_and_l = fit_final.evaluate(test)

In [55]:
p_and_l.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[100,101,102...|[8.64987620236763...|[0.99982488214184...|       0.0|
|  0.0|(692,[122,123,124...|[19.1972970238896...|[0.99999999540040...|       0.0|
|  0.0|(692,[122,123,148...|[21.3606746784008...|[0.99999999947133...|       0.0|
|  0.0|(692,[124,125,126...|[21.3696065646204...|[0.99999999947604...|       0.0|
|  0.0|(692,[124,125,126...|[21.6177786924033...|[0.99999999959119...|       0.0|
|  0.0|(692,[126,127,128...|[17.6517856431645...|[0.99999997842618...|       0.0|
|  0.0|(692,[128,129,130...|[20.0015634595043...|[0.99999999794206...|       0.0|
|  0.0|(692,[151,152,153...|[28.7246586071413...|[0.99999999999966...|       0.0|
|  0.0|(692,[153,154,155...|[26.6116491563972...|[0.99999999999722...|       0.0|
|  0.0|(692,[154

In [59]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
my_eval = BinaryClassificationEvaluator()


In [60]:
my_final_roc = my_eval.evaluate(p_and_l.predictions)

In [62]:
my_final_roc #perfect fit

1.0

In [63]:
evaluator1 = MulticlassClassificationEvaluator()
f1 = evaluator1.evaluate(p_and_l.predictions)
f1

1.0